In [ ]:
!pip install skfeature-chappers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 1.6 MB/s eta 0:00:00


In [ ]:
fileName = 'Dataset.csv'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest,chi2,RFECV
from skfeature.function.similarity_based import fisher_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss

In [ ]:
df=pd.read_csv(fileName,index_col='Unnamed: 0')
df

,x_1,x_109,x_113,x_12,x_124,x_127,x_144,x_16,x_18,x_2,...,x_73,x_77,x_78,x_8,x_80,x_83,x_89,x_91,x_99,y
0,5099,1,5,156,35,-2,1,40,999,7,...,0.11,629,14,487,-0.31,115,0.02,-0.10,-0.49,0
1,5191,0,1,164,16,1,5,4,999,5,...,0.11,509,26,346,-0.31,317,0.02,0.24,-0.21,0
2,5228,1,0,774,75,1,5,12,999,4,...,0.11,1058,30,227,-0.31,464,0.02,-0.10,-0.21,0
3,5228,1,1,747,4,1,5,69,999,3,...,0.11,1048,4,17,-0.31,39,-0.18,-0.10,-0.21,0
4,5196,1,5,29,28,0,4,32,999,6,...,0.11,993,22,58,-0.31,492,0.02,-0.10,0.21,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4114,5228,1,5,197,82,1,5,6,999,4,...,0.11,672,23,53,-0.31,474,-0.07,-0.10,0.21,0
4115,5228,1,5,949,72,1,5,2,999,6,...,0.11,1058,6,219,-0.31,475,0.02,-0.10,0.21,0
4116,5099,1,1,478,41,-2,1,25,999,1,...,0.11,1240,16,64,0.35,210,0.02,0.24,0.93,0
4117,5228,1,0,554,36,1,5,68,999,0,...,0.11,1152,21,528,-0.31,49,0.02,-0.10,0.21,0


In [ ]:
df.columns
Y=df['y']
X=df.drop("y",axis=1)
Xn=X

In [ ]:
#Normalizing the data
for i in X.columns:
    X.loc[:,i]=(X.loc[:,i]-X.loc[:,i].mean())
    X.loc[:,i]=X.loc[:,i]/X.loc[:,i].std()
X

,x_1,x_109,x_113,x_12,x_124,x_127,x_144,x_16,x_18,x_2,...,x_71,x_73,x_77,x_78,x_8,x_80,x_83,x_89,x_91,x_99
0,-0.916845,0.880479,0.766841,-1.192330,-0.536467,-1.277891,-1.429598,0.253080,0.201007,0.692525,...,0.968569,0.492143,-1.032039,-0.262445,0.903842,-0.307921,-0.918287,0.445348,-0.596733,-1.188139
1,0.333350,-1.135470,-0.805198,-1.164881,-1.194757,0.764388,0.762089,-1.508038,0.201007,0.002346,...,-1.032201,0.492143,-1.325876,0.993569,0.350258,-0.307921,0.472444,0.445348,1.579401,-0.452148
2,0.836146,0.880479,-1.198208,0.928070,0.849407,0.764388,0.762089,-1.116678,0.201007,-0.342744,...,-1.032201,0.492143,0.018428,1.412240,-0.116952,-0.307921,1.484511,0.445348,-0.596733,-0.452148
3,0.836146,0.880479,-0.805198,0.835431,-1.610519,0.764388,0.762089,1.671757,0.201007,-0.687833,...,0.968569,0.492143,-0.006059,-1.309122,-0.941439,-0.307921,-1.441532,-4.092406,-0.596733,-0.452148
4,0.401295,0.880479,0.766841,-1.628075,-0.778995,0.083628,0.214167,-0.138280,0.201007,0.347435,...,-1.032201,0.492143,-0.140734,0.574898,-0.780468,-0.307921,1.677285,0.445348,-0.596733,0.651838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4114,0.836146,0.880479,0.766841,-1.051656,1.091935,0.764388,0.762089,-1.410198,0.201007,-0.342744,...,0.968569,0.492143,-0.926748,0.679566,-0.800098,-0.307921,1.553359,-1.596641,-0.596733,0.651838
4115,0.836146,0.880479,0.766841,1.528507,0.745466,0.764388,0.762089,-1.605878,0.201007,0.347435,...,0.968569,0.492143,0.018428,-1.099787,-0.148361,-0.307921,1.560244,0.445348,-0.596733,0.651838
4116,-0.916845,0.880479,-0.805198,-0.087526,-0.328586,-1.277891,-1.429598,-0.480719,0.201007,-1.378013,...,-1.032201,0.492143,0.464080,-0.053109,-0.756911,0.845675,-0.264230,0.445348,1.579401,2.544386
4117,0.836146,0.880479,-1.198208,0.173235,-0.501820,0.764388,0.762089,1.622837,0.201007,-1.723102,...,-1.032201,0.492143,0.248600,0.470230,1.064813,-0.307921,-1.372684,0.445348,-0.596733,0.651838


In [ ]:
final=[]

#Filter Method

##Correlation Coefficient

In [ ]:
cd=X.corr()
plt.figure(figsize=(48,32))
sns.set(font_scale=1.4)
sns.heatmap(cd.abs(),cmap="Reds",annot=True,annot_kws={"size":15},vmax=0.07,vmin=0.025)

NameError: name 'X' is not defined

###From the above correlation, we can observe that

####['x_1', 'x_127', 'x_144', 'x_18', 'x_25', 'x_27', 'x_28', 'x_38','x_45', 'x_52','x_57','x_60','x_73','x_80', 'x_91', 'x_99'] are the confirmed features that we got after setting the values of vmin to 0.2,0.025,0.05 and 0.035 due to their high correlation coefficient values with a good number of other selected features.

#### 4 features out of these probable features ['x_68' ,'x_55','x_71',X_8,x_109,x_89] are the remaining required features

##Chi-Square Test

In [ ]:
# Select 20 optimal features
k_best = 20
selector = SelectKBest(score_func=chi2, k=k_best)
X_selected = selector.fit_transform(abs(X), Y)
selected_indices = selector.get_support(indices=True)
selected_features = X.columns[selected_indices]

# Display the selected features
print("Selected Features using Chi-Square:")
print(selected_features)
final.extend(selected_features)

Selected Features using Chi-Square:
Index(['x_1', 'x_127', 'x_144', 'x_18', 'x_25', 'x_27', 'x_28', 'x_33', 'x_45',
       'x_52', 'x_55', 'x_57', 'x_60', 'x_73', 'x_8', 'x_80', 'x_83', 'x_89',
       'x_91', 'x_99'],
      dtype='object')


##Fisher's Score

In [ ]:
scores = fisher_score.fisher_score(Xn.values,Y.values)

# Get the indices of the top features
sorted_indices = np.argsort(scores)
selected_indices = scores[sorted_indices[-20:]]

# Get the names of the selected features
selected_features = X.columns[selected_indices]

# Display the selected features
print("Selected Features using Fisher's Score:")
print(selected_features)
final.extend(selected_features)

Selected Features using Fisher's Score:
Index(['x_55', 'x_57', 'x_59', 'x_60', 'x_62', 'x_64', 'x_67', 'x_68', 'x_7',
       'x_70', 'x_71', 'x_73', 'x_77', 'x_78', 'x_8', 'x_80', 'x_83', 'x_89',
       'x_91', 'x_99'],
      dtype='object')


#Wrapper Method

##Recursive Elimination with Cross Validation



In [ ]:
model = LogisticRegression()
# Initialize RFE with the logistic regression model
rfecv = RFECV(model, scoring="accuracy",min_features_to_select=20)

X_train_rfecv = rfecv.fit_transform(X,Y)

# Get the indices of the selected features
selected_indices = rfecv.get_support(indices=True)

# Get the names of the selected features
selected_features = X.columns[selected_indices]

# Display the selected features
print("Selected Features using Recursive Feature Elimination:")
print(selected_features)
final.extend(selected_features)

Selected Features using Recursive Feature Elimination:
Index(['x_1', 'x_109', 'x_127', 'x_144', 'x_16', 'x_18', 'x_2', 'x_25', 'x_27',
       'x_28', 'x_33', 'x_38', 'x_42', 'x_44', 'x_45', 'x_52', 'x_57', 'x_60',
       'x_62', 'x_64', 'x_67', 'x_68', 'x_71', 'x_73', 'x_77', 'x_78', 'x_8',
       'x_80', 'x_83', 'x_89', 'x_91', 'x_99'],
      dtype='object')


##Frequency of probable features

In [ ]:
from collections import Counter
frequency=Counter(final)
# Sort the frequency dictionary by values in descending order
sorted_frequency = dict(sorted(frequency.items(), key=lambda item: item[1], reverse=True))

# Print the sorted frequency
print("Sorted Frequency in Descending Order:")
print(sorted_frequency)


Sorted Frequency in Descending Order:
{'x_57': 3, 'x_60': 3, 'x_73': 3, 'x_8': 3, 'x_80': 3, 'x_83': 3, 'x_89': 3, 'x_91': 3, 'x_99': 3, 'x_1': 2, 'x_127': 2, 'x_144': 2, 'x_18': 2, 'x_25': 2, 'x_27': 2, 'x_28': 2, 'x_33': 2, 'x_45': 2, 'x_52': 2, 'x_55': 2, 'x_62': 2, 'x_64': 2, 'x_67': 2, 'x_68': 2, 'x_71': 2, 'x_77': 2, 'x_78': 2, 'x_59': 1, 'x_7': 1, 'x_70': 1, 'x_109': 1, 'x_16': 1, 'x_2': 1, 'x_38': 1, 'x_42': 1, 'x_44': 1}


####From the Chi-Square Test,Fisher's Score, Dispersion Ratio and Recursive elimination it is evident that out of the 6 probable features.... x_8 is the most suitable one and x_109 is the most unsuitable one...

####But from['x_55','x_71','x_68','x_89'], all these four have passed atleast 2 out of the 4 tests.

####So we need to pic 3 out these 4 featues, for which we can try using Logistic Regression Model to know the accuracy as per the features we input and select the best possible features.

#Estimated Features

In [ ]:
X_train1=df[['x_1','x_127', 'x_144','x_18','x_25', 'x_27', 'x_28','x_45', 'x_52','x_57','x_60', 'x_73','x_8','x_80', 'x_91', 'x_99','x_38','x_89','x_55','x_68']]
X_train2=df[['x_1','x_127', 'x_144','x_18','x_25', 'x_27', 'x_28','x_45', 'x_52','x_57','x_60', 'x_73','x_8','x_80', 'x_91', 'x_99','x_38','x_89','x_71','x_68']]
X_train3=df[['x_1','x_127', 'x_144','x_18','x_25', 'x_27', 'x_28','x_45', 'x_52','x_57','x_60', 'x_73','x_8','x_80', 'x_91', 'x_99','x_38','x_89','x_55','x_71']]
X_train4=df[['x_1','x_127', 'x_144','x_18','x_25', 'x_27', 'x_28','x_45', 'x_52','x_57','x_60', 'x_73','x_8','x_80', 'x_91', 'x_99','x_38','x_71','x_55','x_68']]

#Checking Accuracy


##Logistic Regression

###X_train1

In [ ]:


# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train1, Y, test_size=0.2, random_state=42)



# Initialize Logistic Regression model
model = LogisticRegression(random_state=42, max_iter=1000)

# Train the model
model.fit(X_train, y_train)

# Predictions on the training set
train_predictions = model.predict(X_train)
train_probabilities = model.predict_proba(X_train)

# Calculate and print training loss (log loss) and accuracy
train_loss = log_loss(y_train, train_probabilities)
train_accuracy = accuracy_score(y_train, train_predictions)
print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")

# Predictions on the test set
test_predictions = model.predict(X_test)
test_probabilities = model.predict_proba(X_test)

# Calculate and print testing loss (log loss) and accuracy
test_loss = log_loss(y_test, test_probabilities)
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Loss: {test_loss:.4f}")
print(f"Testing Accuracy: {test_accuracy * 100:.2f}%")


Training Loss: 0.1988
Training Accuracy: 91.78%
Testing Loss: 0.2178
Testing Accuracy: 89.93%


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


###X_train2

In [ ]:


# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train2, Y, test_size=0.2, random_state=42)



# Initialize Logistic Regression model
model = LogisticRegression(random_state=42, max_iter=1000)

# Train the model
model.fit(X_train, y_train)

# Predictions on the training set
train_predictions = model.predict(X_train)
train_probabilities = model.predict_proba(X_train)

# Calculate and print training loss (log loss) and accuracy
train_loss = log_loss(y_train, train_probabilities)
train_accuracy = accuracy_score(y_train, train_predictions)
print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")

# Predictions on the test set
test_predictions = model.predict(X_test)
test_probabilities = model.predict_proba(X_test)

# Calculate and print testing loss (log loss) and accuracy
test_loss = log_loss(y_test, test_probabilities)
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Loss: {test_loss:.4f}")
print(f"Testing Accuracy: {test_accuracy * 100:.2f}%")


Training Loss: 0.2006
Training Accuracy: 91.84%
Testing Loss: 0.2143
Testing Accuracy: 90.41%


###X_train3

In [ ]:


# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train3, Y, test_size=0.2, random_state=42)



# Initialize Logistic Regression model
model = LogisticRegression(random_state=42, max_iter=1000)

# Train the model
model.fit(X_train, y_train)

# Predictions on the training set
train_predictions = model.predict(X_train)
train_probabilities = model.predict_proba(X_train)

# Calculate and print training loss (log loss) and accuracy
train_loss = log_loss(y_train, train_probabilities)
train_accuracy = accuracy_score(y_train, train_predictions)
print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")

# Predictions on the test set
test_predictions = model.predict(X_test)
test_probabilities = model.predict_proba(X_test)

# Calculate and print testing loss (log loss) and accuracy
test_loss = log_loss(y_test, test_probabilities)
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Loss: {test_loss:.4f}")
print(f"Testing Accuracy: {test_accuracy * 100:.2f}%")


Training Loss: 0.1984
Training Accuracy: 92.08%
Testing Loss: 0.2194
Testing Accuracy: 89.93%


###X_train4

In [ ]:


# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train4, Y, test_size=0.2, random_state=42)



# Initialize Logistic Regression model
model = LogisticRegression(random_state=42, max_iter=1000)

# Train the model
model.fit(X_train, y_train)

# Predictions on the training set
train_predictions = model.predict(X_train)
train_probabilities = model.predict_proba(X_train)

# Calculate and print training loss (log loss) and accuracy
train_loss = log_loss(y_train, train_probabilities)
train_accuracy = accuracy_score(y_train, train_predictions)
print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")

# Predictions on the test set
test_predictions = model.predict(X_test)
test_probabilities = model.predict_proba(X_test)

# Calculate and print testing loss (log loss) and accuracy
test_loss = log_loss(y_test, test_probabilities)
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Loss: {test_loss:.4f}")
print(f"Testing Accuracy: {test_accuracy * 100:.2f}%")


Training Loss: 0.1983
Training Accuracy: 92.02%
Testing Loss: 0.2188
Testing Accuracy: 90.05%


####All the four data set seem to work well in the above model

#Final Choice


In [ ]:
X_train1=df[['x_1','x_127', 'x_144','x_18','x_25', 'x_27', 'x_28','x_45', 'x_52','x_57','x_60', 'x_73','x_8','x_80', 'x_91', 'x_99','x_38','x_89','x_55','x_68']]


In [ ]:
cols=['x_1','x_127', 'x_144','x_18','x_25', 'x_27', 'x_28','x_45', 'x_52','x_57','x_60', 'x_73','x_8','x_80', 'x_91', 'x_99','x_38','x_68','x_89','x_55']

In [ ]:
cols2=df.drop(columns=cols).drop(columns=['y']).columns.values
cols2

array(['x_109', 'x_113', 'x_12', 'x_124', 'x_16', 'x_2', 'x_30', 'x_33',
       'x_42', 'x_44', 'x_59', 'x_62', 'x_64', 'x_67', 'x_7', 'x_70',
       'x_71', 'x_77', 'x_78', 'x_83'], dtype=object)

In [ ]:
df_final2=pd.DataFrame()
df_final2['Variables']=cols2
df_final2['Mark']=0
df_final2

,Variables,Mark
0,x_109,0
1,x_113,0
2,x_12,0
3,x_124,0
4,x_16,0
5,x_2,0
6,x_30,0
7,x_33,0
8,x_42,0
9,x_44,0


In [ ]:
df_final=pd.DataFrame()
df_final['Variables']=cols

In [ ]:
df_final['Mark']=1

In [ ]:
df_final

,Variables,Mark
0,x_1,1
1,x_127,1
2,x_144,1
3,x_18,1
4,x_25,1
5,x_27,1
6,x_28,1
7,x_45,1
8,x_52,1
9,x_57,1


In [ ]:
for i in cols2:
  print(df[i].value_counts())

1    2320
0    1799
Name: x_109, dtype: int64
5    1055
6    1043
1    1025
0     996
Name: x_113, dtype: int64
356    13
854    12
860    12
284    11
420    10
       ..
352     1
484     1
611     1
346     1
564     1
Name: x_12, Length: 990, dtype: int64
37     55
45     55
60     52
99     51
3      50
       ..
17     32
7      31
74     28
100    19
0      16
Name: x_124, Length: 101, dtype: int64
59    83
66    77
6     73
43    72
32    72
      ..
16    47
8     46
60    45
0     34
70    31
Name: x_16, Length: 71, dtype: int64
3     452
6     434
2     429
9     424
1     415
4     408
8     406
5     393
7     379
10    202
0     177
Name: x_2, dtype: int64
0    2085
1    2034
Name: x_30, dtype: int64
1.10     6
8.95     5
7.77     5
0.71     5
0.58     5
        ..
19.35    1
47.37    1
42.99    1
59.47    1
74.67    1
Name: x_33, Length: 3038, dtype: int64
4     157
7     151
13    150
28    147
18    147
29    146
16    143
26    143
3     143
23    142
15    142
11    

In [ ]:
for i in cols:
  print(df[df['y']==1][i].value_counts())

5099    91
5228    89
5076    57
4992    45
5018    39
5009    38
4964    35
5191    25
5196    20
5024    12
Name: x_1, dtype: int64
-2    174
 1    114
-3    108
-1     35
 0     20
Name: x_127, dtype: int64
1    304
5    117
4     17
2     13
Name: x_144, dtype: int64
999    351
3       39
6       27
7        8
10       6
4        5
5        4
12       3
0        2
15       1
18       1
19       1
9        1
21       1
2        1
Name: x_18, dtype: int64
31    27
33    25
30    22
32    22
38    17
41    17
39    16
50    14
28    13
37    13
29    13
36    13
27    12
54    12
48    11
34    10
58    10
44    10
52    10
26    10
45     9
47     9
51     9
35     8
59     8
57     8
25     7
53     7
43     7
42     7
49     6
60     6
55     6
24     4
40     4
80     3
56     3
64     3
21     3
66     3
46     3
61     3
81     2
22     2
23     2
75     2
71     2
68     2
62     2
86     2
76     2
82     2
63     1
69     1
77     1
78     1
67     1
73     1
18     1
74     

In [ ]:
for i in cols:
  print(df[df['y']==0][i].value_counts())

5228    1537
5191     733
5099     732
5196     372
5076     107
5018      65
4964      48
4992      42
5009      22
5024       9
5176       1
Name: x_1, dtype: int64
 1    2270
-2     796
 0     373
-3     181
-1      48
Name: x_127, dtype: int64
5    2275
1    1013
4     367
2      12
3       1
Name: x_144, dtype: int64
999    3608
6        15
3        13
4         9
1         3
2         3
10        2
7         2
12        2
16        2
9         2
13        2
18        1
11        1
15        1
17        1
14        1
Name: x_18, dtype: int64
32    194
34    164
31    164
35    164
30    155
     ... 
77      1
19      1
69      1
80      1
81      1
Name: x_25, Length: 64, dtype: int64
94    1845
93    1689
92     107
95      27
Name: x_27, dtype: int64
-36    1238
-42     777
-43     624
-46     547
-47     174
-31      51
-41      45
-40      38
-38      37
-30      37
-27      28
-50      25
-51      14
-35      13
-34      11
-33       9
Name: x_28, dtype: int64
 0.29    2277


In [ ]:
df_final3=pd.concat([df_final, df_final2], ignore_index=True)

In [ ]:
df_final3.reset_index(drop=True, inplace=True)

In [ ]:
df_final3.to_csv('TENSILE TENSORS_IIT MADRAS_VAR.csv',index=False)

##Making the csv for methods

In [ ]:
methods= ['Correlation Coefficient','Chi-Square Test','Fishers Score','Recursive Elimination with Cross Validation']

In [ ]:
df_methods=pd.DataFrame()
df_methods['methods']=methods
df_methods

,methods
0,Correlation Coefficient
1,Chi-Square Test
2,Fishers Score
3,Recursive Elimination with Cross Validation


In [ ]:
df_methods.to_csv('TENSILE TENSORS_IIT MADRAS_METHOD.csv',index=False)